In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from scipy.spatial import distance

In [9]:
#info = pd.read_csv('patient_information.csv')
info = pd.read_csv('/Users/dylanmack/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/ESE 527/EMR/patient_information.csv')
info.head()

PID  Age   Ht  Wt Gender        OR_start          OR_end  \
0  56d1bc3e2a07cc1a   19  160  51      F     1/1/16 7:30    1/1/16 15:25   
1  411a637540c1d665   51  162  71      F   12/26/16 7:11   12/26/16 8:32   
2  51206705422a4c5c   41  165  58      F    6/19/16 8:15   6/19/16 13:37   
3  40750c9577f139bd   55  162  66      F  12/11/15 13:13  12/11/15 20:09   
4  ed4ac97e460b5a79   41  158  76      F    4/18/16 7:11    4/18/16 9:31   

    Surgery_start     Surgery_end  \
0     1/1/16 8:29    1/1/16 15:14   
1   12/26/16 7:50   12/26/16 8:21   
2    6/19/16 8:57   6/19/16 13:37   
3  12/11/15 14:07  12/11/15 19:51   
4    4/18/16 8:07    4/18/16 9:31   

                                           Procedure  
0  Funtional rhinoplasty, repair of vestibular st...  
1  Excision Ganglion Cyst left dorsal hand,splint...  
2        Abdominoplasty, Bilateral Flank Liposuction  
3  1.Bilateral endoscopic sinus surgery, inferior...  
4  Left breast: Tissue expander Exchange for Sili...

In [10]:
info_new = info[['PID']].copy()
info_new['Age'] = info['Age'].apply(lambda x: int(x))
info_new['Ht'] = info['Ht'].apply(lambda x: int(x) if x != '\\N' else 0)
info_new['Wt'] = info['Wt'].apply(lambda x: int(x))
info_new.loc[:,'Sex'] = info.loc[:,'Gender'].apply(lambda x: 1 if x == 'F' else 0)
info_new.loc[:,'OR_time'] = (pd.to_datetime(info.loc[:,'OR_end'])-pd.to_datetime(info.loc[:,'OR_start'])).apply(lambda x: x.total_seconds()/60)
info_new.loc[:,'Surgery_time'] = (pd.to_datetime(info.loc[:,'Surgery_end'])-pd.to_datetime(info.loc[:,'Surgery_start'])).apply(lambda x: x.total_seconds()/60)
info_new.loc[:,'Procedure'] = info.loc[:,'Procedure']

info_new = info_new[info_new['Ht'] != 0]
info_new = info_new[info_new['OR_time'] < 10000]

info_new.head()

PID  Age   Ht  Wt  Sex  OR_time  Surgery_time  \
0  56d1bc3e2a07cc1a   19  160  51    1    475.0         405.0   
1  411a637540c1d665   51  162  71    1     81.0          31.0   
2  51206705422a4c5c   41  165  58    1    322.0         280.0   
3  40750c9577f139bd   55  162  66    1    416.0         344.0   
4  ed4ac97e460b5a79   41  158  76    1    140.0          84.0   

                                           Procedure  
0  Funtional rhinoplasty, repair of vestibular st...  
1  Excision Ganglion Cyst left dorsal hand,splint...  
2        Abdominoplasty, Bilateral Flank Liposuction  
3  1.Bilateral endoscopic sinus surgery, inferior...  
4  Left breast: Tissue expander Exchange for Sili...

In [15]:
info_new['age_group'] = info_new['Age'].apply(lambda x: 'child' if x < 19 else ('elder' if x > 64 else 'adult'))
info_new

PID  Age   Ht   Wt  Sex  OR_time  Surgery_time  \
0      56d1bc3e2a07cc1a   19  160   51    1    475.0         405.0   
1      411a637540c1d665   51  162   71    1     81.0          31.0   
2      51206705422a4c5c   41  165   58    1    322.0         280.0   
3      40750c9577f139bd   55  162   66    1    416.0         344.0   
4      ed4ac97e460b5a79   41  158   76    1    140.0          84.0   
...                 ...  ...  ...  ...  ...      ...           ...   
19109  039e1d2687af1657   32  175   61    0    426.0         362.0   
19110  978fb2f756398ad9   45  162   69    1    243.0         181.0   
19111  fdac376288499893   29  185   88    0    171.0         125.0   
19112  27f74474129a0c63   62  162   72    1    220.0         173.0   
19113  aca232a2eb2d82af   55  182  108    0    631.0         547.0   

                                               Procedure age_group  
0      Funtional rhinoplasty, repair of vestibular st...     adult  
1      Excision Ganglion Cyst left dorsal hand,splint...     adult  
2            Abdominoplasty, Bilateral Flank Liposuction     adult  
3      1.Bilateral endoscopic sinus surgery, inferior...     adult  
4      Left breast: Tissue expander Exchange for Sili...     adult  
...                                                  ...       ...  
19109  Open displaced transcondylar fracture of right...     adult  
19110  RE-EXPLORATION RIGHT RETROSIGMOID VRANIECTOMY ...     adult  
19111  Removal of hardware right lower extremity with...     adult  
19112  Cystoscopy, right retrograde pyelogram, right ...     adult  
19113  Cystoscopy; Suprapubic Tube Exchange; Urethral...     adult  

[18987 rows x 9 columns]